In [1]:
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import os
import json
import math
import numpy as np
from scipy import signal
from scipy import sparse
from skimage.util import img_as_float
from scipy import io as scio
from scipy import linalg
from sklearn.metrics import mean_squared_error


In [2]:
def detrend(input_signal, lambda_value):
    signal_length = input_signal.shape[0]
    # observation matrix
    H = np.identity(signal_length)
    ones = np.ones(signal_length)
    minus_twos = -2 * np.ones(signal_length)
    diags_data = np.array([ones, minus_twos, ones])
    diags_index = np.array([0, 1, 2])
    D = sparse.spdiags(diags_data, diags_index,
                (signal_length - 2), signal_length).toarray()
    filtered_signal = np.dot(
        (H - np.linalg.inv(H + (lambda_value ** 2) * np.dot(D.T, D))), input_signal)
    return filtered_signal


# def process_video(frames):
#     RGB = []
#     for frame in frames:
#         summation = np.sum(np.sum(frame, axis=0), axis=0)
#         RGB.append(summation / (frame.shape[0] * frame.shape[1]))
#     RGB = np.asarray(RGB)
#     RGB = RGB.transpose(1, 0).reshape(1, 3, -1)
#     return np.asarray(RGB)
def process_video(frames):
    RGB = []
    for frame in frames:
        summation = np.sum(np.sum(frame, axis=0), axis=0)
        RGB.append(summation / (frame.shape[0] * frame.shape[1]))
    RGB = np.asarray(RGB)

    # Check if there's only one frame
    if RGB.shape[0] == 1:
        RGB = RGB.transpose(1, 0).reshape(3, -1)
    else:
        RGB = RGB.transpose(1, 0).reshape(1, 3, -1)

    return np.asarray(RGB)


In [3]:
"""ICA
Non-contact, automated cardiac pulse measurements using video imaging and blind source separation.
Poh, M. Z., McDuff, D. J., & Picard, R. W. (2010).
Optics express, 18(10), 10762-10774. DOI: 10.1364/OE.18.010762
"""

def ICA_POH(frames, FS):
    # Cut off frequency.
    LPF = 0.7
    HPF = 2.5
    RGB = process_video(frames)

    NyquistF = 1 / 2 * FS
    BGRNorm = np.zeros(RGB.shape)
    Lambda = 100
    for c in range(3):
        BGRDetrend = detrend(RGB[:, c], Lambda)
        BGRNorm[:, c] = (BGRDetrend - np.mean(BGRDetrend)) / np.std(BGRDetrend)
    _, S = ica(np.mat(BGRNorm).H, 3)

    # select BVP Source
    MaxPx = np.zeros((1, 3))
    for c in range(3):
        FF = np.fft.fft(S[c, :])
        F = np.arange(0, FF.shape[1]) / FF.shape[1] * FS * 60
        FF = FF[:, 1:]
        FF = FF[0]
        N = FF.shape[0]
        Px = np.abs(FF[:math.floor(N / 2)])
        Px = np.multiply(Px, Px)
        Fx = np.arange(0, N / 2) / (N / 2) * NyquistF
        Px = Px / np.sum(Px, axis=0)
        MaxPx[0, c] = np.max(Px)
    MaxComp = np.argmax(MaxPx)
    BVP_I = S[MaxComp, :]
    B, A = signal.butter(3, [LPF / NyquistF, HPF / NyquistF], 'bandpass')
    BVP_F = signal.filtfilt(B, A, np.real(BVP_I).astype(np.double))

    BVP = BVP_F[0]
    return BVP


def process_video(frames):
    "Calculates the average value of each frame."
    RGB = []
    for frame in frames:
        sum = np.sum(np.sum(frame, axis=0), axis=0)
        RGB.append(sum / (frame.shape[0] * frame.shape[1]))
    return np.asarray(RGB)


def ica(X, Nsources, Wprev=0):
    nRows = X.shape[0]
    nCols = X.shape[1]
    if nRows > nCols:
        print(
            "Warning - The number of rows is cannot be greater than the number of columns.")
        print("Please transpose input.")

    if Nsources > min(nRows, nCols):
        Nsources = min(nRows, nCols)
        print(
            'Warning - The number of soures cannot exceed number of observation channels.')
        print('The number of sources will be reduced to the number of observation channels ', Nsources)

    Winv, Zhat = jade(X, Nsources, Wprev)
    W = np.linalg.pinv(Winv)
    return W, Zhat


def jade(X, m, Wprev):
    n = X.shape[0]
    T = X.shape[1]
    nem = m
    seuil = 1 / math.sqrt(T) / 100
    if m < n:
        D, U = np.linalg.eig(np.matmul(X, np.mat(X).H) / T)
        Diag = D
        k = np.argsort(Diag)
        pu = Diag[k]
        ibl = np.sqrt(pu[n - m:n] - np.mean(pu[0:n - m]))
        bl = np.true_divide(np.ones(m, 1), ibl)
        W = np.matmul(np.diag(bl), np.transpose(U[0:n, k[n - m:n]]))
        IW = np.matmul(U[0:n, k[n - m:n]], np.diag(ibl))
    else:
        IW = linalg.sqrtm(np.matmul(X, X.H) / T)
        W = np.linalg.inv(IW)

    Y = np.mat(np.matmul(W, X))
    R = np.matmul(Y, Y.H) / T
    C = np.matmul(Y, Y.T) / T
    Q = np.zeros((m * m * m * m, 1))
    index = 0

    for lx in range(m):
        Y1 = Y[lx, :]
        for kx in range(m):
            Yk1 = np.multiply(Y1, np.conj(Y[kx, :]))
            for jx in range(m):
                Yjk1 = np.multiply(Yk1, np.conj(Y[jx, :]))
                for ix in range(m):
                    Q[index] = np.matmul(Yjk1 / math.sqrt(T), Y[ix, :].T / math.sqrt(
                        T)) - R[ix, jx] * R[lx, kx] - R[ix, kx] * R[lx, jx] - C[ix, lx] * np.conj(C[jx, kx])
                    index += 1
    # Compute and Reshape the significant Eigen
    D, U = np.linalg.eig(Q.reshape(m * m, m * m))
    Diag = abs(D)
    K = np.argsort(Diag)
    la = Diag[K]
    M = np.zeros((m, nem * m), dtype=complex)
    Z = np.zeros(m)
    h = m * m - 1
    for u in range(0, nem * m, m):
        Z = U[:, K[h]].reshape((m, m))
        M[:, u:u + m] = la[h] * Z
        h = h - 1
    # Approximate the Diagonalization of the Eigen Matrices:
    B = np.array([[1, 0, 0], [0, 1, 1], [0, 0 - 1j, 0 + 1j]])
    Bt = np.mat(B).H

    encore = 1
    if Wprev == 0:
        V = np.eye(m).astype(complex)
    else:
        V = np.linalg.inv(Wprev)
    # Main Loop:
    while encore:
        encore = 0
        for p in range(m - 1):
            for q in range(p + 1, m):
                Ip = np.arange(p, nem * m, m)
                Iq = np.arange(q, nem * m, m)
                g = np.mat([M[p, Ip] - M[q, Iq], M[p, Iq], M[q, Ip]])
                temp1 = np.matmul(g, g.H)
                temp2 = np.matmul(B, temp1)
                temp = np.matmul(temp2, Bt)
                D, vcp = np.linalg.eig(np.real(temp))
                K = np.argsort(D)
                la = D[K]
                angles = vcp[:, K[2]]
                if angles[0, 0] < 0:
                    angles = -angles
                c = np.sqrt(0.5 + angles[0, 0] / 2)
                s = 0.5 * (angles[1, 0] - 1j * angles[2, 0]) / c

                if abs(s) > seuil:
                    encore = 1
                    pair = [p, q]
                    G = np.mat([[c, -np.conj(s)], [s, c]])  # Givens Rotation
                    V[:, pair] = np.matmul(V[:, pair], G)
                    M[pair, :] = np.matmul(G.H, M[pair, :])
                    temp1 = c * M[:, Ip] + s * M[:, Iq]
                    temp2 = -np.conj(s) * M[:, Ip] + c * M[:, Iq]
                    temp = np.concatenate((temp1, temp2), axis=1)
                    M[:, Ip] = temp1
                    M[:, Iq] = temp2

    # Whiten the Matrix
    # Estimation of the Mixing Matrix and Signal Separation
    A = np.matmul(IW, V)
    S = np.matmul(np.mat(V).H, Y)
    return A, S

In [4]:
def POS_WANG(frames, fs):
    WinSec = 1.6
    RGB = process_video(frames)
    N = RGB.shape[0]
    H = np.zeros((1, N))
    l = math.ceil(WinSec * fs)

    for n in range(N):
        m = n - l
        if m >= 0:
            Cn = np.true_divide(RGB[m:n, :], np.mean(RGB[m:n, :], axis=0))
            Cn = np.mat(Cn).H
            S = np.matmul(np.array([[0, 1, -1], [-2, 1, 1]]), Cn)
            h = S[0, :] + (np.std(S[0, :]) / np.std(S[1, :])) * S[1, :]
            mean_h = np.mean(h)
            for temp in range(h.shape[1]):
                h[0, temp] = h[0, temp] - mean_h
            H[0, m:n] = H[0, m:n] + (h[0])

    BVP = H
    BVP = detrend(np.mat(BVP).H, 100)
    BVP = np.asarray(np.transpose(BVP))[0]
    b, a = signal.butter(1, [0.75 / fs * 2, 3 / fs * 2], btype='bandpass')
    BVP = signal.filtfilt(b, a, BVP.astype(np.double))
    return BVP

def CHROME_DEHAAN(frames,FS):
    LPF = 0.7
    HPF = 2.5
    WinSec = 1.6

    RGB = process_video(frames)
    FN = RGB.shape[0]
    NyquistF = 1/2*FS
    B, A = signal.butter(3, [LPF/NyquistF, HPF/NyquistF], 'bandpass')

    WinL = math.ceil(WinSec*FS)
    if(WinL % 2):
        WinL = WinL+1
    NWin = math.floor((FN-WinL//2)/(WinL//2))
    WinS = 0
    WinM = int(WinS+WinL//2)
    WinE = WinS+WinL
    totallen = (WinL//2)*(NWin+1)
    S = np.zeros(totallen)

    for i in range(NWin):
        RGBBase = np.mean(RGB[WinS:WinE, :], axis=0)
        RGBNorm = np.zeros((WinE-WinS, 3))
        for temp in range(WinS, WinE):
            RGBNorm[temp-WinS] = np.true_divide(RGB[temp], RGBBase)
        Xs = np.squeeze(3*RGBNorm[:, 0]-2*RGBNorm[:, 1])
        Ys = np.squeeze(1.5*RGBNorm[:, 0]+RGBNorm[:, 1]-1.5*RGBNorm[:, 2])
        Xf = signal.filtfilt(B, A, Xs, axis=0)
        Yf = signal.filtfilt(B, A, Ys)

        Alpha = np.std(Xf) / np.std(Yf)
        SWin = Xf-Alpha*Yf
        SWin = np.multiply(SWin, np.hanning(WinL))

        temp = SWin[:int(WinL//2)]
        S[WinS:WinM] = S[WinS:WinM] + SWin[:int(WinL//2)]
        S[WinM:WinE] = SWin[int(WinL//2):]
        WinS = WinM
        WinM = WinS+WinL//2
        WinE = WinS+WinL
    BVP = S
    return BVP


In [5]:
def _load_and_preprocess_video(video_path, batch_size=20):
    path = 'haarcascade_frontalface_default.xml'
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + path)

    face_frames = []
    batch = []

    while True:
        ret, frame = video_path.read()
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces:
            face_frame = frame[y:y + h, x:x + w]
            face_frame = cv2.resize(face_frame, (240, 240))
            face_frame = face_frame / 255.0
            batch.append(face_frame)

            if len(batch) == batch_size:
                face_frames.append(np.array(batch))
                batch = []

    if batch:  # If there are remaining frames in the last batch
        face_frames.append(np.array(batch))

    video_path.release()

    return np.concatenate(face_frames, axis=0)


In [6]:
# import numpy as np
# class HealthDataset(Dataset):
#     def __init__(self, root_dir, transform=None):
#         self.root_dir = root_dir
#         self.samples = self._load_samples()
#         self.transform = transform

#     def _load_samples(self):
#         samples = []
#         for dirpath, _, filenames in os.walk(self.root_dir):
#             for filename in filenames:
#                 if filename.endswith('.json'):
#                     json_path = os.path.join(dirpath, filename)
#                     with open(json_path) as json_file:
#                         data = json.load(json_file)
#                         for scenario in data['scenarios']:
#                             scenario_settings = scenario['scenario_settings']
#                             if (scenario_settings['position'] == "Sitting" and
#                                     scenario_settings['facial_movement'] == "No movement" and
#                                     scenario_settings['talking'] == "N"):
#                                 rgb_filename = scenario['recordings']['RGB']['filename']
#                                 if rgb_filename in filenames:
#                                     sample = {
#                                         "GUID": data['GUID'],
#                                         "participant_metadata": {**data['participant'], 'GUID': data['GUID']},
#                                         "ppg_values": [item[1] for item in scenario['recordings']['ppg']['timeseries']],
#                                         "recording_link": os.path.join(dirpath, rgb_filename)
#                                     }
#                                     if 'bp_sys' in scenario['recordings'] and 'bp_dia' in scenario['recordings']:
#                                         sample["bp_values"] = {
#                                             "bp_sys": scenario['recordings']['bp_sys']['value'],
#                                             "bp_dia": scenario['recordings']['bp_dia']['value'],
#                                         }
#                                     samples.append(sample)
#         return samples

#     def process_and_save_ica_output(self, save_path='output.csv', batch_size=300):
#           with open(save_path, 'w') as csv_file:
#               csv_file.write("GUID,Gender,Age,PPG,ICA_Output,CHROME_Output,POS_Output,BP_Sys,BP_Dia\n")  # Header
#               total_samples = len(self.samples)
#               for idx, sample in enumerate(self.samples):
#                   # Load and preprocess the video frames
#                   cap = cv2.VideoCapture(sample['recording_link'])
#                   num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

#                   ica_output_batches = []
#                   chrome_output_batches = []
#                   pos_output_batches = []

#                   for i in range(0, num_frames, batch_size):
#                       frames_batch = []
#                       for _ in range(batch_size):
#                           ret, frame = cap.read()
#                           if not ret:
#                               break
#                           frames_batch.append(frame)
#                       frames_batch = np.array(frames_batch)

#                       # Pass facial frames through the ICA function
#                       ica_output_batches.append(ICA_POH(frames_batch, FS=30))

#                       # Pass facial frames through the CHROME_DEHAAN function
#                       chrome_output_batches.append(CHROME_DEHAAN(frames_batch, FS=30))

#                       # Pass facial frames through the POS_WANG function
#                       pos_output_batches.append(POS_WANG(frames_batch, fs=30))

#                   ica_output = np.concatenate(ica_output_batches)
#                   chrome_output = np.concatenate(chrome_output_batches)
#                   pos_output = np.concatenate(pos_output_batches)

#                   # Convert the output arrays to string representations
#                   ica_output_str = ",".join(map(str, ica_output))
#                   chrome_output_str = ",".join(map(str, chrome_output))
#                   pos_output_str = ",".join(map(str, pos_output))

#                   # Write GUID, Gender, Age, PPG, ICA output, CHROME output, POS output, BP Sys, and BP Dia to the CSV file
#                   csv_file.write(f"{sample['GUID']},{sample['participant_metadata']['gender']},{sample['participant_metadata']['age']},{','.join(map(str, sample['ppg_values']))},{ica_output_str},{chrome_output_str},{pos_output_str},{sample.get('bp_values', {}).get('bp_sys', '')},{sample.get('bp_values', {}).get('bp_dia', '')}\n")

#                   progress_percentage = (idx + 1) / total_samples * 100
#                   print(f"\rProcessing: {progress_percentage:.2f}% complete", end='', flush=True)

#               cap.release()

#     def __getitem__(self, idx):
#         sample = self.samples[idx]

#         # Load and preprocess the video frames
#         video_frames = self._load_and_preprocess_video(cv2.VideoCapture(sample['recording_link']))

#         # Convert data to PyTorch tensors
#         ppg_values = torch.tensor(sample['ppg_values'], dtype=torch.float32)

#         if 'bp_values' in sample:
#             bp_values = torch.tensor([sample['bp_values']['bp_sys'], sample['bp_values']['bp_dia']], dtype=torch.float32)
#             return {'ppg': ppg_values, 'video_frames': video_frames, 'bp': bp_values}

#         return {'ppg': ppg_values, 'video_frames': video_frames, 'ica_output': torch.tensor(ICA_POH(video_frames, FS=30)),
#                 'chrome_output': torch.tensor(CHROME_DEHAAN(video_frames, FS=30)),
#                 'pos_output': torch.tensor(POS_WANG(video_frames, FS=30))}
#     def __len__(self):
#             return len(self.samples)


In [7]:
import os
import json
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset
from scipy import signal

class HealthDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.samples = self._load_samples()
        self.transform = transform

    def _load_samples(self):
        samples = []
        for dirpath, _, filenames in os.walk(self.root_dir):
            for filename in filenames:
                if filename.endswith('.json'):
                    json_path = os.path.join(dirpath, filename)
                    with open(json_path) as json_file:
                        data = json.load(json_file)
                        for scenario in data['scenarios']:
                            scenario_settings = scenario['scenario_settings']
                            if (scenario_settings['position'] == "Sitting" and
                                    scenario_settings['facial_movement'] == "No movement" and
                                    scenario_settings['talking'] == "N"):
                                rgb_filename = scenario['recordings']['RGB']['filename']
                                if rgb_filename in filenames:
                                    sample = {
                                        "GUID": data['GUID'],
                                        "participant_metadata": {**data['participant'], 'GUID': data['GUID']},
                                        "ppg_values": [item[1] for item in scenario['recordings']['ppg']['timeseries']],
                                        "recording_link": os.path.join(dirpath, rgb_filename)
                                    }
                                    if 'bp_sys' in scenario['recordings'] and 'bp_dia' in scenario['recordings']:
                                        sample["bp_values"] = {
                                            "bp_sys": scenario['recordings']['bp_sys']['value'],
                                            "bp_dia": scenario['recordings']['bp_dia']['value'],
                                        }
                                    samples.append(sample)
        return samples
    def process_and_save_ica_output(self, save_path='output.csv', batch_size=300):
        with open(save_path, 'w') as csv_file:
            csv_file.write("GUID,Gender,Age,PPG,ICA_Output,CHROME_Output,POS_Output,BP_Sys,BP_Dia\n")  # Header
            total_samples = len(self.samples)
            for idx, sample in enumerate(self.samples):
                # Load and preprocess the video frames
                cap = cv2.VideoCapture(sample['recording_link'])
                num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                print(f"Total number of frames: {num_frames}")  # Print total number of frames

                ica_output_batches = []
                chrome_output_batches = []
                pos_output_batches = []

                for i in range(0, num_frames, batch_size):
                    frames_batch = []
                    for _ in range(batch_size):
                        ret, frame = cap.read()
                        if not ret:
                            break
                        frames_batch.append(frame)
                    if frames_batch:  # Check if frames_batch is not empty
                        frames_batch = np.array(frames_batch)

                        # Pass facial frames through the ICA function
                        ica_output_batches.append(ICA_POH(frames_batch, FS=30))

                        # Pass facial frames through the CHROME_DEHAAN function
                        chrome_output_batches.append(CHROME_DEHAAN(frames_batch, FS=30))

                        # Pass facial frames through the POS_WANG function
                        pos_output_batches.append(POS_WANG(frames_batch, fs=30))

                if ica_output_batches:
                    ica_output = np.concatenate(ica_output_batches)
                else:
                    ica_output = np.array([])

                if chrome_output_batches:
                    chrome_output = np.concatenate(chrome_output_batches)
                else:
                    chrome_output = np.array([])

                if pos_output_batches:
                    pos_output = np.concatenate(pos_output_batches)
                else:
                    pos_output = np.array([])

                # Convert the output arrays to string representations
                ica_output_str = ','.join(map(str, ica_output))
                chrome_output_str = ','.join(map(str, chrome_output))
                pos_output_str = ','.join(map(str, pos_output))

                # Write GUID, Gender, Age, PPG, ICA output, CHROME output, POS output, BP Sys, and BP Dia to the CSV file
                # csv_file.write(f"{sample['GUID']},{sample['participant_metadata']['gender']},{sample['participant_metadata']['age']},{','.join(map(str, sample['ppg_values']))},{ica_output_str},{chrome_output_str},{pos_output_str},{sample.get('bp_values', {}).get('bp_sys', '')},{sample.get('bp_values', {}).get('bp_dia', '')}\n")
                #csv_file.write(f"{sample['GUID']},{sample['participant_metadata']['gender']},{sample['participant_metadata']['age']},{','.join(map(str, sample['ppg_values']))},{','.join(map(str, ica_output))},{','.join(map(str, chrome_output))},{','.join(map(str, pos_output))},{sample.get('bp_values', {}).get('bp_sys', '')},{sample.get('bp_values', {}).get('bp_dia', '')}\n")
                # csv_file.write(f"{sample['GUID']},{sample['participant_metadata']['gender']},{sample['participant_metadata']['age']},[{','.join(map(str, sample['ppg_values']))}],[{','.join(map(str, ica_output))}],[{','.join(map(str, chrome_output))}],[{','.join(map(str, pos_output))}],{sample.get('bp_values', {}).get('bp_sys', '')},{sample.get('bp_values', {}).get('bp_dia', '')}\n")
                csv_file.write(f"{sample['GUID']},{sample['participant_metadata']['gender']},{sample['participant_metadata']['age']},\"[{','.join(map(str, sample['ppg_values']))}]\",\"[{','.join(map(str, ica_output))}]\",\"[{','.join(map(str, chrome_output))}]\",\"[{','.join(map(str, pos_output))}]\",{sample.get('bp_values', {}).get('bp_sys', '')},{sample.get('bp_values', {}).get('bp_dia', '')}\n")

                progress_percentage = (idx + 1) / total_samples * 100
                print(f"\rProcessing: {progress_percentage:.2f}% complete", end='', flush=True)

                cap.release()


    def __getitem__(self, idx):
        sample = self.samples[idx]

        # Load and preprocess the video frames
        video_frames = self._load_and_preprocess_video(cv2.VideoCapture(sample['recording_link']))

        # Convert data to PyTorch tensors
        ppg_values = torch.tensor(sample['ppg_values'], dtype=torch.float32)

        if 'bp_values' in sample:
            bp_values = torch.tensor([sample['bp_values']['bp_sys'], sample['bp_values']['bp_dia']], dtype=torch.float32)
            return {'ppg': ppg_values, 'video_frames': video_frames, 'bp': bp_values}

        return {'ppg': ppg_values, 'video_frames': video_frames, 'ica_output': torch.tensor(np.concatenate(ICA_POH(video_frames, FS=30)), dtype=torch.float32),
                'chrome_output': torch.tensor(np.concatenate(CHROME_DEHAAN(video_frames, FS=30)), dtype=torch.float32),
                'pos_output': torch.tensor(np.concatenate(POS_WANG(video_frames, FS=30)), dtype=torch.float32)}

    def __len__(self):
        return len(self.samples)

# Your ICA_POH, CHROME_DEHAAN, and POS_WANG functions would be defined here




In [9]:
# Example usage:
root_dir = 'D:/Dataset/VV/vv100'
health_dataset = HealthDataset(root_dir)
health_dataset.process_and_save_ica_output(save_path='OUT1.csv')


Total number of frames: 900
Processing: 0.50% completeTotal number of frames: 900
Processing: 1.00% completeTotal number of frames: 900
Processing: 1.49% completeTotal number of frames: 900
Processing: 1.99% completeTotal number of frames: 900
Processing: 2.49% completeTotal number of frames: 900
Processing: 2.99% completeTotal number of frames: 900
Processing: 3.48% completeTotal number of frames: 900
Processing: 3.98% completeTotal number of frames: 900
Processing: 4.48% completeTotal number of frames: 900
Processing: 4.98% completeTotal number of frames: 900
Processing: 5.47% completeTotal number of frames: 900
Processing: 5.97% completeTotal number of frames: 900
Processing: 6.47% completeTotal number of frames: 0
Processing: 6.97% completeTotal number of frames: 900
Processing: 7.46% completeTotal number of frames: 900
Processing: 7.96% completeTotal number of frames: 900
Processing: 8.46% completeTotal number of frames: 900
Processing: 8.96% completeTotal number of frames: 900
Pr

KeyboardInterrupt: 